https://www.goflightlabs.com

In [1]:
ACCESS_KEY = 'ACCESS_KEY'

In [2]:
import requests
import json
from pprint import pprint
from datetime import datetime, timedelta

import pandas as pd
import numpy as np

import plotly.express as px

from tqdm import tqdm

In [3]:
link_flights = 'https://app.goflightlabs.com/flights'
link_airlines = 'https://app.goflightlabs.com/airlines'
link_airports = 'https://app.goflightlabs.com/airports'
link_airport_data = 'https://app.goflightlabs.com/get-airport-data'
link_countries = 'https://app.goflightlabs.com/countries'

date = '2023-01-01'
link_historical = f'https://app.goflightlabs.com/historical/{date}'

In [4]:
def get_response(link, params, is_logger=True):
    response = requests.get(link, params)
    if is_logger:
        print(response)
    
    return response

## Get Countries

In [5]:
# create params
params = {
    'access_key': ACCESS_KEY
}
# get countries params
response = get_response(link_countries, params)
response_json = response.json()

# collect list of European Union countries
eu_coutries_list = []
for country in response_json['data']:
    # EU countries are located on European continent and have EUR currency
    if country['codeCurrency'] == 'EUR' and country['continent'] == 'EU':
        eu_coutries_list.append(country)

<Response [200]>


In [13]:
print(f'Count of EU countries: {len(eu_coutries_list)}')

Count of EU countries: 24


In [15]:
print('EU countries response example')
eu_coutries_list[12]

EU countries response example


{'capital': 'Rome',
 'codeCurrency': 'EUR',
 'codeFips': 'IT',
 'codeIso2Country': 'IT',
 'codeIso3Country': 'ITA',
 'continent': 'EU',
 'countryId': 112,
 'nameCountry': 'Italy',
 'nameCurrency': 'Euro',
 'numericIso': '380',
 'phonePrefix': '39',
 'population': '60340328'}

In [8]:
df_countries = pd.DataFrame(eu_coutries_list)
df_countries.head()

,capital,codeCurrency,codeFips,codeIso2Country,codeIso3Country,continent,countryId,nameCountry,nameCurrency,numericIso,phonePrefix,population
0,Andorra la Vella,EUR,AN,AD,AND,EU,1,Andorra,Euro,20,376,84000
1,Vienna,EUR,AU,AT,AUT,EU,13,Austria,Euro,40,43,8205000
2,Mariehamn,EUR,,AX,ALA,EU,16,Aland Islands,Euro,248,+358-18,26711
3,Brussels,EUR,BE,BE,BEL,EU,21,Belgium,Euro,56,32,10403000
4,Nicosia,EUR,CY,CY,CYP,EU,57,Cyprus,Euro,196,357,1102677


## Get Airports

In [17]:
# create params
params = {
    'access_key': ACCESS_KEY,
}

# collect airports params by each country
airports_data_dict_list = []
for country in eu_coutries_list:
    # add country code to params
    params['codeIso2Country'] = country['codeIso2Country']
    # get airports params
    response = get_response(link_airports, params)
    response_json = response.json()
    # check if airports by the country were found
    if response_json['success']:
        response_json_data = response_json['data']
        airports_data_dict_list.append(response_json_data)

# union all the airports' responses data to the common dataframe
for i in range(len(airports_data_dict_list)):
    if i == 0:
        df_airports = pd.DataFrame(airports_data_dict_list[i])
    else:
        df_airports = pd.concat([df_airports, pd.DataFrame(airports_data_dict_list[i])])

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [25]:
print(f'Count of EU airports: {df_airports.count()["airportId"]}')

Count of EU airports: 806


## Get Flights

- Select a start date
- Select a period end date (+3-5 days from start_date) 
- Per each airport:
    - Collect all the arrival flights to the selected airport
- Save the data to JSON

In [8]:
def get_next_date(date, delta):
    next_datetime = datetime.strptime(date, '%Y-%m-%d') + timedelta(days=delta)
    next_date = datetime.strftime(next_datetime, '%Y-%m-%d')
    return next_date

In [17]:
get_next_date('2023-01-25', 105)

'2023-05-10'

In [18]:
# Link
link_historical = 'https://app.goflightlabs.com/historical/'
# List of airports
df_airports_list = df_airports['codeIataAirport'].tolist()
# Parsing period
start_date = '2023-01-25'
end_date = get_next_date(start_date, 105)

# Main Loop by dates
while start_date <= end_date:
    # Each loop iteration = 3 days period
    date_to = get_next_date(start_date, 3)
    # Create params dict
    params = {
        'access_key': ACCESS_KEY,
        'type': 'arrival',
        'date_to': date_to
    }
    # Collect arrival floghts for each airport
    airport_response_list = []
    for codeIataAirport in tqdm(df_airports_list):
        params['code'] = codeIataAirport
        # Get response data
        response = get_response(link_historical+start_date, params, False)
        response_json = response.json()
        # Check response data
        if response_json['success']:
            # Collect response data
            airport_response_list.append(response_json['data'])
    # Save middle data to JSON
    with open(f'flights_arrival_data/{start_date}.json', 'w') as f:
        json.dump(airport_response_list, f)
    # Init new start date
    print(start_date)
    start_date = get_next_date(date_to, 1)

100%|█████████████████████████████████████████| 806/806 [12:18<00:00,  1.09it/s]


2023-01-25


100%|█████████████████████████████████████████| 806/806 [21:09<00:00,  1.57s/it]


2023-01-29


100%|█████████████████████████████████████████| 806/806 [21:01<00:00,  1.57s/it]


...

2023-05-01


100%|█████████████████████████████████████████| 806/806 [22:13<00:00,  1.65s/it]


2023-05-05


100%|█████████████████████████████████████████| 806/806 [22:25<00:00,  1.67s/it]


2023-05-09
